In [311]:
import numpy as np
import pandas as pd

In [294]:
import math

def sigmoid(x):
    if x < 0:
        return 1 - 1 / (1 + math.exp(x))
    return 1 / (1 + math.exp(-x))

# Fungsi sigmoid untuk numpy array
sigmoid_v = np.vectorize(sigmoid)

In [295]:
class Dense:
    def __init__(self, layers, activation):
        self.layers = layers
        self.activation = activation

In [296]:
class Sequential:
    def __init__(self, input_shape, denses):
        self.denses = denses
        self.input_shape = input_shape
        self.weights = []

    def set_weight(self):
        last_dense = 0
        for i, dense in enumerate(self.denses):
            weight = []
            if i == 0:
                # weight.append(np.random.randn(self.input_shape, dense.layers))
                weight.append(np.random.randint(-100, 100, (self.input_shape, dense.layers)) * 0.01)
            else:
                # weight.append(np.random.randn(last_dense, dense.layers))
                weight.append(np.random.randint(-100, 100, (last_dense, dense.layers)) * 0.01)
                
            bias = []
            # bias.append(np.random.randn(1, dense.layers))
            bias.append(np.random.randint(-100, 100, (1, dense.layers)) * 0.01)

            last_dense = dense.layers

            self.weights.append(weight)
            self.weights.append(bias)

    def get_weight(self):
        return self.weights
    
    def feedForward(self, X_train, y_train):
        # print("feedforward")
        mat = []

        it = iter(self.weights)
        for i, w in enumerate(it):
            bias = next(it)

            if (isinstance(w, np.ndarray)):
                w = w.tolist()
            if (isinstance(bias, np.ndarray)):
                bias = bias.tolist()

            if i == 0:
                result = np.matmul(X_train, w) + bias
            else:
                result = np.matmul(mat[-1], w) + bias

            result = np.squeeze(result, axis=0)
            mat.append(result)

        # print(mat)
        sig = []
        for m in mat:
            m = sigmoid_v(m)
            sig.append(m)

        mat = sig
            
        err = 0.5 * np.square(y_train - mat[-1])
 
        return mat, err
    
    
    def backPropagation(self, X_train, y_train, mat):
        # print("backpropagation")
        delta = []
        for i, m in enumerate(reversed(mat)):
            if i == 0:
                delta.append(m * (1 - m) * (y_train - m))
            else:
                if (isinstance(self.weights, np.ndarray)):
                    self.weights = self.weights.tolist()

                part_1 = m * (1 - m)
                
                part_2 = np.transpose(self.weights[len(self.weights) - (i * 2)])
                
                if (len(part_2.shape) == 3):
                    part_2 = np.squeeze(part_2, axis=2)

                part_3 = np.dot(delta[-1], part_2)

                if (len(part_3.shape) == 3):
                    part_3 = np.squeeze(part_3, axis=2)

                result = part_1 * part_3

                if (len(result.shape) == 3):
                    result = np.squeeze(result, axis=0)
                delta.append(result)

        return delta
    
    def updateWeights(self, X_train, y_train, mat, delta, momentum, learning_rate, last_weight):
        # print("updateWeights")
        new_weights = []

        it = iter(reversed(self.weights))

        for i, w in enumerate(it):
            bias = w
            weight = next(it)
            
            if (i == len(mat) - 1):
                new_weight = learning_rate * (X_train.reshape(X_train.shape[0], 1) @ delta[i])
                new_bias = learning_rate * delta[i]
            else:
                new_weight = learning_rate * (np.transpose(mat[len(mat) - i - 2]) @ delta[i])
                new_bias = learning_rate * delta[i]
            
            if (len(new_weight.shape) == 3):
                new_weight = np.squeeze(new_weight, axis=0)

            if (momentum):
                if last_weight:
                    new_weight += momentum * np.squeeze(np.asarray(weight), axis=0)
                    new_bias += momentum * np.squeeze(np.asarray(bias), axis=0)
                else:
                    new_weight += momentum * last_weight[len(last_weight) - (i * 2)]
                    new_bias += momentum * last_weight[len(last_weight) - (i * 2) + 1]
            
            bias = np.asarray(bias)
            bias = np.squeeze(bias, axis=0)
            new_weights.insert(0, new_bias + bias)
            new_weights.insert(0, new_weight + np.squeeze(np.asarray(weight), axis=0))

        new_weights = np.asarray(new_weights)
        new_weights = new_weights.reshape(new_weights.shape[0], 1)
        
        self.weights = new_weights
    
    def fit(self, X_train, y_train, epochs, momentum, learning_rate):
        self.set_weight()

        list_err = []
        last_weight = []

        for epoch in range(epochs):
            for i in range(len(X_train)):
                mat, err = self.feedForward(X_train[i], y_train[i])
                delta = self.backPropagation(X_train[i], y_train[i], mat)
                last_weight = self.weights

                self.updateWeights(X_train[i], y_train, mat, delta, momentum, learning_rate, last_weight)                

                list_err.append(err[0][0])
        
        return mat
            
    def predict(self, X_test):
        prediction = []
        list_err = []

        for X in X_test:
            mat = []

            it = iter(self.weights)
            for i, w in enumerate(it):
                bias = next(it)
                
                if i == 0:
                    X = X.astype(float)
                    X = X.reshape(1, self.input_shape)

                    result = np.dot(X, w[0]) + bias[0]
                    mat.append(result)
                else:
                    result = np.dot(mat[-1], w[0]) + bias[0]
                    mat.append(result)
            
            sig = []
            for m in mat:
                m = sigmoid_v(m)
                sig.append(m)
            mat = sig
                
            err = 0.5 * np.square(y_train - mat[-1])
            list_err.append(err[0][0])

            # print(mat[-1])
            pred = np.greater(mat[-1], 0.5)
            prediction.append(pred)
            
        prediction = np.asarray(prediction).astype(int)
        prediction = prediction.squeeze(axis=1)
        prediction = prediction.squeeze(axis=1)

        return prediction, list_err

## Test pada make_blob dataset

In [303]:
from sklearn.datasets import make_blobs
X_train, y_train = make_blobs(n_samples=50, centers=2, n_features=2)

# print(X_train[0].shape)

In [306]:
model = Sequential(2, [
    Dense(3, 'sigmoid'),
    Dense(2, 'sigmoid'),
    Dense(1, 'sigmoid')
])

mat = model.fit(X_train, y_train, epochs=100, momentum=0.001, learning_rate=0.02)
prediction, list_err = model.predict(X_train)

print(prediction)
print(y_train)
a = 0
b = len(prediction)
for i, j in zip(prediction, y_train):
    if (i == j):
        a += 1
print(a/b)

[1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1
 0 1 0 1 0 0 0 1 0 1 1 0 1]
[1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1
 0 1 0 1 0 0 0 1 0 1 1 0 1]
1.0


## Test pada weather dataset

In [314]:
from scipy.io.arff import loadarff

with open('weather.arff') as f:
    data, meta = loadarff(f)
    
datas = []
for i in data:
    d = []
    # print(i)
    for j in i:
        # print(j)
        d.append(j)
    datas.append(d)
    
datas = np.asarray(datas)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for i in data:
    # data[i][0] = le.fit_transform(data[i][0])
    datas[:, 0] = le.fit_transform(datas[:, 0])
    datas[:, 3] = le.fit_transform(datas[:, 3])
    datas[:, 4] = le.fit_transform(datas[:, 4])
    
X_train = datas[:, :-1]
y_train = datas[:, -1:]

X_train = X_train.astype(float)
y_train = y_train.astype(int)
y_train = y_train.squeeze(axis=1)
# X_train = preprocessing.normalize(X_train)

# print(X_train)
x_normed = X_train / X_train.max(axis=0)
# print(x_normed)
X_train = x_normed
# print(y_train)

In [329]:
model = Sequential(4, [
    Dense(3, 'sigmoid'),
    Dense(2, 'sigmoid'),
    Dense(1, 'sigmoid')
])

mat = model.fit(X_train, y_train, epochs=100, momentum=0.001, learning_rate=0.02)
prediction, list_err = model.predict(X_train)

print(prediction)
print(y_train)
a = 0
b = len(prediction)
for i, j in zip(prediction, y_train):
    if (i == j):
        a += 1
print(a/b)

[0 0 1 1 0 0 1 0 0 1 0 1 1 1]
[0 0 1 1 1 0 1 0 1 1 1 1 1 0]
0.7142857142857143
